<a href="https://colab.research.google.com/github/Piyushaa7/OptimaScan/blob/main/CTScanEnhancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras matplotlib gradio opencv-python

In [ ]:
import os
import zipfile
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import gradio as gr

In [ ]:
# Unzip trainA.zip
with zipfile.ZipFile("/content/trainA.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/trainA")

# Unzip testA.zip
with zipfile.ZipFile("/content/testA.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/testA")


In [ ]:
!ls /content

sample_data  testA.zip	trainA.zip


In [ ]:
import zipfile

print(zipfile.is_zipfile("/content/trainA.zip"))  # Should be True
print(zipfile.is_zipfile("/content/testA.zip"))   # Should be True

True
True


In [ ]:
def load_images_from_folder(folder, img_size=(128,128)):
    images = []
    for root, dirs, files in os.walk(folder):
        for filename in files:
            if filename.endswith(".png"):
                img = cv2.imread(os.path.join(root, filename), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
    images = np.array(images, dtype='float32') / 255.0
    images = np.expand_dims(images, axis=-1)
    return images

X_train = load_images_from_folder("/content/trainA")
X_test = load_images_from_folder("/content/testA")

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (1742, 128, 128, 1)
Test set shape: (744, 128, 128, 1)


In [ ]:
def add_noise(images, noise_factor=0.3):
    noisy = images + noise_factor * np.random.randn(*images.shape)
    noisy = np.clip(noisy, 0., 1.)
    return noisy

X_train_noisy = add_noise(X_train)
X_test_noisy = add_noise(X_test)

In [ ]:
def build_autoencoder(input_shape=(128,128,1)):
    input_img = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2,2))(x)
    decoded = layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

autoencoder = build_autoencoder()
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 1)    │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,497 (291.00 KB)

 Trainable params: 74,497 (291.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
autoencoder.fit(
    X_train_noisy, X_train,
    epochs=40,
    batch_size=16,
    shuffle=True,
    validation_data=(X_test_noisy, X_test)
)


Epoch 1/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - loss: 0.0571 - val_loss: 0.0131
Epoch 2/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - loss: 0.0119 - val_loss: 0.0099
Epoch 3/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - loss: 0.0093 - val_loss: 0.0085
Epoch 4/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - loss: 0.0080 - val_loss: 0.0078
Epoch 5/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - loss: 0.0074 - val_loss: 0.0071
Epoch 6/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - loss: 0.0069 - val_loss: 0.0067
Epoch 7/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.0065 - val_loss: 0.0063
Epoch 8/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - loss: 0.0062 - val_loss: 0.0061
Epoch 9/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - loss: 0.0059 - val_loss: 0.0059
Epoch 10/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.0058 - val_loss: 0.0057
Epoch 11/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - loss: 0.0057 - val_loss: 0.0056
Epoch 12/40
109/109 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
autoencoder.save("autoencoder.h5")

NameError: name 'autoencoder' is not defined

In [ ]:
autoencoder.save("autoencoder.keras")

In [ ]:
import gradio as gr

def denoise_ct(image):
    import cv2
    import numpy as np

    # Resize to 128x128 and normalize
    img = cv2.resize(image, (128,128))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=(0,-1))

    # Predict
    denoised = autoencoder.predict(img)[0,:,:,0]
    denoised = (denoised * 255).astype('uint8')
    return denoised

interface = gr.Interface(
    fn=denoise_ct,
    inputs=gr.Image(type="numpy", image_mode='L'),  # removed 'shape'
    outputs=gr.Image(type="numpy", image_mode='L'), # removed 'shape'
    title="CT Scan Denoiser",
    description="Upload a CT scan image, and the autoencoder will denoise it."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5c99a72095a562bc6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
%%writefile app.py
import gradio as gr

def denoise_ct(image):
    import cv2
    import numpy as np

    # Resize to 128x128 and normalize
    img = cv2.resize(image, (128,128))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=(0,-1))

    # Predict
    denoised = autoencoder.predict(img)[0,:,:,0]
    denoised = (denoised * 255).astype('uint8')
    return denoised

interface = gr.Interface(
    fn=denoise_ct,
    inputs=gr.Image(type="numpy", image_mode='L'),  # removed 'shape'
    outputs=gr.Image(type="numpy", image_mode='L'), # removed 'shape'
    title="CT Scan Denoiser",
    description="Upload a CT scan image, and the autoencoder will denoise it."
)

interface.launch()


Writing app.py


In [ ]:
%%writefile requirements.txt
gradio
opencv-python
numpy
tensorflow

Writing requirements.txt


In [ ]:
!pip install gradio huggingface_hub

PSNR

In [ ]:
# For example, pick the first image
clean_img = X_test[0]         # Ground truth clean image
noisy_img = X_test_noisy[0]   # Noisy input


In [ ]:
denoised_img = autoencoder.predict(np.expand_dims(noisy_img, axis=0))[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step


In [ ]:
import math
import numpy as np

def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1.0  # images normalized between 0 and 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

psnr_score = psnr(denoised_img, clean_img)
print("PSNR:", psnr_score)

PSNR: 25.43761706638302


SSIM

In [ ]:
from skimage.metrics import structural_similarity as ssim

# Ensure images are squeezed (H x W) and normalized between 0 and 1
clean_img_squeezed = clean_img.squeeze()
denoised_img_squeezed = denoised_img.squeeze()

ssim_score = ssim(clean_img_squeezed, denoised_img_squeezed, data_range=1.0)
print("SSIM:", ssim_score)


SSIM: 0.77727628572654


In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Map` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggi

In [ ]:
!git config --global credential.helper store

In [ ]:
!gradio deploy

Creating new Spaces Repo in '/content'. Collecting metadata, press Enter to 
accept default value.
Enter Spaces app title [content]: OptimaScan
Enter Gradio app file [app.py]: app.py
Enter Spaces hardware (cpu-basic, cpu-upgrade, cpu-xl, zero-a10g, t4-small, t4-medium, l4x1, l4x4, l40sx1, l40sx4, l40sx8, a10g-small, a10g-large, a10g-largex2, a10g-largex4, a100-large, h100, h100x8) [cpu-basic]: cpu-basic
Any Spaces secrets (y/n) [n]: n
Create Github Action to automatically update Space on 'git push'? [n]: n
It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`hf upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            
New Data Upload     